# benchmark

> A module to benchmark Pytorch model according to: size, speed, compute and energy.

In [ ]:
#| default_exp benchmark

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
import time
from codecarbon import OfflineEmissionsTracker
import numpy as np
import os
from thop import profile, clever_format
from tqdm.notebook import tqdm
from prettytable import PrettyTable
from torchprofile import profile_macs
import torch.nn as nn
from typing import Dict, List, Tuple, Any, Union, Optional

## Size

In [ ]:
#| export
def get_model_size(model: nn.Module,            # The model we want to study
                   temp_path: str ="temp.pth"   # The temporary path used to save the model
                  ) -> int:
    try: model.save(temp_path)
    except: torch.save(model.state_dict(), temp_path)
    
    model_size = os.path.getsize(temp_path)
    os.remove(temp_path)
    
    return model_size

In [ ]:
#| export
def get_num_parameters(model: nn.Module  # The model we want to study
                      ) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

---

## Speed

In [ ]:
#| export
@torch.inference_mode()
def evaluate_gpu_speed(model:nn.Module,            # The model we want to evaluate
                       dummy_input: torch.Tensor,  # The input used to evaluate the model
                       warmup_rounds:int=50,       # The amount of warmup iterations
                       test_rounds:int=100        # The amount of iterations used to evaluate the speed
                      )-> Tuple[float, float, float]:
    device = torch.device("cuda")
    model.eval()
    model.to(device)
    dummy_input = dummy_input.to(device)
    
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    latencies = []

    # Warm up GPU
    for _ in range(warmup_rounds):
        _ = model(dummy_input)
    
    # Measure Latency
    for _ in range(test_rounds):
        starter.record()
        _ = model(dummy_input)
        ender.record()
        torch.cuda.synchronize()
        latencies.append(starter.elapsed_time(ender))  # time in milliseconds
    
    latencies = np.array(latencies)
    mean_latency = np.mean(latencies)
    std_latency = np.std(latencies)

    # Measure Throughput
    throughput = dummy_input.size(0) * 1000 / mean_latency  # Inferences per second

    return mean_latency, std_latency, throughput

In [ ]:
#| export
@torch.inference_mode()
def evaluate_cpu_speed(model:nn.Module,            # The model we want to evaluate
                       dummy_input: torch.Tensor,  # The input used to evaluate the model
                       warmup_rounds:int=50,       # The amount of warmup iterations
                       test_rounds:int=100        # The amount of iterations used to evaluate the speed
                      )-> Tuple[float, float, float]:
    device = torch.device("cpu")
    model.eval()
    model.to(device)
    dummy_input = dummy_input.to(device)
    
    # Warm up CPU
    for _ in range(warmup_rounds):
        _ = model(dummy_input)
    
    # Measure Latency
    latencies = []
    for _ in range(test_rounds):
        start_time = time.perf_counter()
        _ = model(dummy_input)
        end_time = time.perf_counter()
        latencies.append(end_time - start_time)
    
    latencies = np.array(latencies) * 1000  # Convert to milliseconds
    mean_latency = np.mean(latencies)
    std_latency = np.std(latencies)

    # Measure Throughput
    throughput = dummy_input.size(0) * 1000 / mean_latency  # Inferences per second

    return mean_latency, std_latency, throughput

---

## Compute

In [ ]:
#| export
@torch.inference_mode()
def get_model_macs(model,  # The model we want to evaluate
                   inputs  # The input used to evaluate
                  ) -> int:
    return profile_macs(model, inputs)

In [ ]:
#| export
@torch.inference_mode()
def evaluate_gpu_memory_usage(model:nn.Module,             # The model we want to evaluate
                              dummy_input:torch.Tensor,       # The input used to evaluate
                              warmup_rounds:int=10,  # The amount of warmup iterations
                              test_rounds:int=100    # The amount of iterations used to evaluate the speed
                             )-> Tuple[float, float]:
    device = torch.device("cuda")
    model.eval()
    model.to(device)
    dummy_input = dummy_input.to(device)
    
    # Warm up GPU
    for _ in range(warmup_rounds):
        _ = model(dummy_input)
    
    # Measure Memory Usage
    memory_usages = []
    for _ in range(test_rounds):
        torch.cuda.reset_peak_memory_stats(device)
        _ = model(dummy_input)
        torch.cuda.synchronize()
        memory_usages.append(torch.cuda.memory_allocated(device))
    
    memory_usages = np.array(memory_usages)
    average_memory_usage = np.mean(memory_usages)
    peak_memory_usage = torch.cuda.max_memory_allocated(device)
    
    return average_memory_usage, peak_memory_usage

## Energy

In [ ]:
#| export
@torch.inference_mode()
def evaluate_emissions(model:nn.Module,            # The model we want to evaluate
                       dummy_input: torch.Tensor,  # The input used to evaluate the model
                       warmup_rounds:int=50,       # The amount of warmup iterations
                       test_rounds:int=100        # The amount of iterations used to evaluate the speed
                      )-> Tuple[float, float]:
    device = torch.device("cuda")
    model.eval()
    model.to(device)
    dummy_input = dummy_input.to(device)

    # Warm up GPU
    for _ in range(warmup_rounds):
        _ = model(dummy_input)
    
    # Measure Latency
    tracker = OfflineEmissionsTracker(country_iso_code="USA")
    tracker.start()
    for _ in range(test_rounds):
        _ = model(dummy_input)
    tracker.stop()
    total_emissions = tracker.final_emissions
    total_energy_consumed = tracker.final_emissions_data.energy_consumed
    
    # Calculate average emissions and energy consumption per inference
    average_emissions_per_inference = total_emissions / test_rounds
    average_energy_per_inference = total_energy_consumed / test_rounds
    
    return average_emissions_per_inference, average_energy_per_inference

In [ ]:
#| export
@torch.inference_mode()
def benchmark(model:nn.Module,          # The model we want to benchmark
              dummy_input:torch.Tensor  # The input used to benchmark the model
             )-> Dict[str, Union[float, str]]:
    # Model Size
    disk_size = get_model_size(model)
    #num_parameters = get_num_parameters(model)
    
    # GPU Speed
    gpu_latency, gpu_std_latency, gpu_throughput = evaluate_gpu_speed(model, dummy_input)
    
    # CPU Speed
    cpu_latency, cpu_std_latency, cpu_throughput = evaluate_cpu_speed(model, dummy_input)
    
    # Model MACs
    #macs = get_model_macs(model, dummy_input)
    macs, params = profile(model, inputs=(dummy_input, ))
    macs, num_parameters = clever_format([macs, params], "%.3f")
    
    # GPU Memory Usage
    avg_gpu_memory, peak_gpu_memory = evaluate_gpu_memory_usage(model, dummy_input)
    
    # Emissions
    avg_emissions, avg_energy = evaluate_emissions(model, dummy_input)
    
    # Print results
    print(f"Model Size: {disk_size / 1e6:.2f} MB (disk), {num_parameters} parameters")
    print(f"GPU Latency: {gpu_latency:.3f} ms (± {gpu_std_latency:.3f} ms)")
    print(f"GPU Throughput: {gpu_throughput:.2f} inferences/sec")
    print(f"CPU Latency: {cpu_latency:.3f} ms (± {cpu_std_latency:.3f} ms)")
    print(f"CPU Throughput: {cpu_throughput:.2f} inferences/sec")
    print(f"Model MACs: {macs}")
    print(f"Average GPU Memory Usage: {avg_gpu_memory / 1e6:.2f} MB")
    print(f"Peak GPU Memory Usage: {peak_gpu_memory / 1e6:.2f} MB")
    print(f"Average Carbon Emissions per Inference: {avg_emissions*1e3:.6f} gCO2e")
    print(f"Average Energy Consumption per Inference: {avg_energy*1e3:.6f} Wh")

    return {

        'disk_size': disk_size,
        'num_parameters': num_parameters, 
        'gpu_latency': gpu_latency, 
        'gpu_throughput': gpu_throughput,
        'cpu_latency': cpu_latency,
        'cpu_throughput': cpu_throughput,
        'macs': macs, 
        'avg_gpu_memory': avg_gpu_memory, 
        'peak_gpu_memory': peak_gpu_memory,
        'avg_emissions': avg_emissions, 
        'avg_energy': avg_energy
        
    }

In [ ]:
show_doc(benchmark)

---

[source](https://github.com/FasterAI-Labs/fasterbench/blob/main/fasterbench/benchmark.py#L179){target="_blank" style="float:right; font-size:smaller"}

### benchmark

>      benchmark (model:torch.nn.modules.module.Module,
>                 dummy_input:torch.Tensor)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| model | Module | The model we want to benchmark |
| dummy_input | Tensor | The input used to benchmark the model |
| **Returns** | **Dict** |  |

In [ ]:
#| export
@torch.inference_mode()
def evaluate(model:nn.Module,   # The module to evaluate
             dls:Any,           # The dataloader used to evaluate 
             device=None,       # The device used to evaluate
             verbose:bool=False # Add some verbose
            )->int:
    if device is None: device = torch.device("cuda")
    model.eval()
    model.to(device)

    with torch.no_grad():
        correct = 0
        total = 0
        local_acc = []
        loader = tqdm(dls.valid, desc="valid", leave=False)
        for i, data in enumerate(loader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0) - 1
            correct += ((predicted.as_subclass(torch.Tensor) == labels.as_subclass(torch.Tensor)).sum().item())

        acc = 100 * correct / total
        if verbose:
            print(f"Valid Accuracy: {acc:.2f} %")
        return acc

In [ ]:
#| export
def compute_model_metrics(model:nn.Module,          # The model we want to evaluate
                          dls:Any,                  # The dataloader used to measure accuracy
                          dummy_input:torch.Tensor # The input used to evaluate the model
                         ) -> Dict[str, Union[float, str]]:
    metrics = {}
    metrics['accuracy'] = round(evaluate(model, dls, device='cpu'), 2)
    metrics['latency'] = round(evaluate_cpu_speed(model.to("cpu"), dummy_input=dummy_input)[0] * 1000, 1)
    metrics['size'] = get_model_size(model)
    try:
        metrics['params'] = round(get_num_parameters(model) / 1e6, 2)
    except RuntimeError:
        metrics['params'] = "*"
    try:
        metrics['mac'] = round(get_model_macs(model, dummy_input) / 1e6)
    except (AttributeError, RuntimeError):
        metrics['mac'] = "*"
    return metrics

In [ ]:
#| export
@torch.inference_mode()
def compare_models(model_list:List[nn.Module], # The list of models we want to compare
                   dls: Any                    # The dataloader used to compare models
                  )-> None:

    metrics_keys = ["latency", "accuracy", "params", "size", "mac"]
    metrics_names = {
        "latency": "Latency (ms/sample)",
        "accuracy": "Accuracy (%)",
        "params": "Params (M)",
        "size": "Size (MiB)",
        "mac": "MACs (M)",
    }
    table_data = {key: [metrics_names[key]] for key in metrics_keys}
    model_names = ["Original Model", "Pruned Model", "Quantized Model"]


    table = PrettyTable()
    table.field_names = [""] + model_names
    table.align = "r"
    table.align[""] = "l"
    table.align["Original Model"] = "l"

    dummy_input = torch.Tensor(next(iter(dls.valid))[0][0][None]).to('cpu')

    model_metrics_list = []
    for model in model_list:
        metrics = compute_model_metrics(model, dls, dummy_input)
        model_metrics_list.append(metrics)

    for metrics in model_metrics_list:
        for key in metrics_keys:
            table_data[key].append(metrics.get(key, "*"))

    for key in metrics_keys:
        values = table_data[key]
        original_value = values[1]
        for i in range(2, len(values)):
            current_value = values[i]
            gain_info = ''
            try:
                orig_val = float(original_value)
                curr_val = float(current_value)
                if key == 'accuracy':
                    gain = curr_val - orig_val
                    gain_info = f'({gain:+.2f}%)'
                else:
                    gain = orig_val / curr_val if curr_val != 0 else float('inf')
                    gain_info = f'({gain:.2f}x)' if gain != float('inf') else '(inf)'
            except (ValueError, TypeError):
                gain_info = ''
            if gain_info:
                values[i] = f'{current_value:<8} {gain_info:>8}'
            else:
                values[i] = f'{current_value}'

    for key in metrics_keys:
        table.add_row(table_data[key])

    print(table)

In [ ]:
show_doc(compare_models)

---

[source](https://github.com/FasterAI-Labs/fasterbench/blob/main/fasterbench/benchmark.py#L281){target="_blank" style="float:right; font-size:smaller"}

### compare_models

>      compare_models (model_list:List[torch.nn.modules.module.Module], dls:Any)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| model_list | List | The list of models we want to compare |
| dls | Any | The dataloader used to compare models |
| **Returns** | **None** |  |